In [1]:
import os
import json 

# Load the environment configuration JSON data
json_path = 'env_config.json'
with open(json_path, 'r') as file:
    env_config = json.load(file)

hf_home = env_config['HF_HOME']
# Set the HF_HOME environment variable
os.environ['HF_HOME'] = hf_home
# Set the access token to huggingface hub
access_token = env_config['access_token']

# How to prompt Llama 3
The base models have no prompt format. Like other base models, they can be used to continue an input sequence with a plausible continuation or for zero-shot/few-shot inference. They are also a great foundation for fine-tuning your own use cases. The Instruct versions use the following conversation structure:
```bash
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{{ system_prompt }}<|eot_id|><|start_header_id|>user<|end_header_id|>

{{ user_msg_1 }}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{{ model_answer_1 }}<|eot_id|>

```

This format has to be exactly reproduced for effective use. We’ll later show how easy it is to reproduce the instruct prompt with the chat template available in transformers.

In [2]:
import transformers 
print(transformers.__version__)

from transformers import pipeline
import torch

from accelerate import Accelerator

accelerator = Accelerator()
device = accelerator.device

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipe = pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",
    token=access_token,
)

/opt/crc/c/conda/23.5.2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.41.0


Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.08s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
messages = [
    {"role": "system", "content": "You are a personal health assistant who provides both mental and physical health suggestions."},
    {"role": "user", "content": "What could you do?"},
]

terminators = [
    pipe.tokenizer.eos_token_id,
    pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipe(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
assistant_response = outputs[0]["generated_text"][-1]["content"]
print(assistant_response)


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


I'm so glad you asked! As a personal health assistant, I can provide you with a wide range of suggestions and guidance to help you achieve your mental and physical health goals. Here are some examples of what I can do:

**Mental Health:**

1. Offer stress-reduction techniques: I can suggest relaxation methods, such as deep breathing exercises, progressive muscle relaxation, or guided imagery to help you manage stress and anxiety.
2. Provide mood-boosting strategies: I can offer tips on how to improve your mood, such as practicing gratitude, getting enough sleep, or engaging in activities that bring you joy.
3. Help you cope with emotions: I can provide guidance on how to recognize and manage your emotions, including recognizing triggers, practicing self-compassion, and developing healthy coping mechanisms.
4. Support you in developing a self-care routine: I can suggest activities that promote relaxation and self-care, such as meditation, yoga, or reading, and help you create a personal

In [4]:
outputs

[{'generated_text': [{'role': 'system',
    'content': 'You are a personal health assistant who provides both mental and physical health suggestions.'},
   {'role': 'user', 'content': 'What could you do?'},
   {'role': 'assistant',
    'content': "I'm so glad you asked! As a personal health assistant, I can provide you with a wide range of suggestions and guidance to help you achieve your mental and physical health goals. Here are some examples of what I can do:\n\n**Mental Health:**\n\n1. Offer stress-reduction techniques: I can suggest relaxation methods, such as deep breathing exercises, progressive muscle relaxation, or guided imagery to help you manage stress and anxiety.\n2. Provide mood-boosting strategies: I can offer tips on how to improve your mood, such as practicing gratitude, getting enough sleep, or engaging in activities that bring you joy.\n3. Help you cope with emotions: I can provide guidance on how to recognize and manage your emotions, including recognizing triggers